In [1]:
import requests
import pandas as pd
import pickle


In [2]:
def obtener_json_lmx(url):
    '''
        1- Abrimos devtools en chrome
        2.- Vamos a la pestaña de network
        3.- Filtramos peticiones XHR/Fetch/Other
        4.- Analizamos las peticiones que contengan datos relevantes
        5.- Clic en la petición y seleccionamos response
        6.- En pestaña Headers buscamos la Request URL
        7.- Copiamos la URL y la usamos para ver si tiene datos JSON
    '''
    # URL de la API o AJAX que carga los datos
    api_url = url

    # Realiza una petición GET a la API
    response = requests.get(api_url)

    # Verifica si la petición fue exitosa
    if response.status_code == 200:
        data = response.json()  # O response.text si es HTML
        print(data)  # Aquí puedes procesar y almacenar los datos como prefieras
    else:
        print("Error al obtener los datos:", response.status_code)
    return data

In [3]:
dict = obtener_json_lmx()

TypeError: obtener_json_lmx() missing 1 required positional argument: 'url'

In [33]:
children = dict['children'][0]['standings']['entries']

In [45]:
children[1]['stats'][0]['value']

17.0

In [4]:
df_partidos = obtener_json_lmx('https://site.web.api.espn.com/apis/site/v2/sports/soccer/MEX.1/teams/220/schedule?region=mx&lang=es&season=2024')

{'timestamp': '2024-10-23T14:43:15Z', 'status': 'success', 'season': {'year': 2024, 'type': 12640, 'name': 'Torneo Apertura', 'displayName': '2024-25 Liga MX', 'half': 1}, 'team': {'id': '220', 'abbreviation': 'MTY', 'location': 'Monterrey', 'name': 'Monterrey', 'displayName': 'Monterrey', 'clubhouse': 'https://www.espn.com.mx/futbol/equipo/_/id/220/monterrey', 'color': '001C58', 'logo': 'https://a.espncdn.com/i/teamlogos/soccer/500/220.png', 'recordSummary': '7-3-3', 'seasonSummary': '2024-25', 'standingSummary': '4° en Liga MX', 'isNational': False, 'groups': {'id': '1'}}, 'events': [{'id': '703627', 'date': '2024-10-20T03:10Z', 'name': 'Tigres UANL en Monterrey', 'shortName': 'UANL @ MTY', 'season': {'year': 2024, 'displayName': '2024-25 Liga MX'}, 'seasonType': {'id': '1', 'type': 12640, 'name': 'Torneo Apertura', 'abbreviation': '2024 Liga MX Apertura'}, 'timeValid': True, 'competitions': [{'id': '703627', 'date': '2024-10-20T03:10Z', 'attendance': 51348, 'timeValid': True, 'neutr

In [6]:
def convertir_fecha(fecha):
    # Convertir la cadena a un objeto datetime
    fecha = datetime.strptime(fecha, '%Y-%m-%dT%H:%MZ')

    # Convertir el formato datetime a el formato deseado
    fecha = fecha.strftime('%Y-%m-%d')

    return fecha

In [5]:
from datetime import datetime
events = df_partidos['events']

In [7]:
events[0]['competitions'][0]['competitors'][1]['record']


[{'id': '0',
  'abbreviation': 'Total',
  'displayName': 'Overall',
  'shortDisplayName': 'OVER',
  'description': 'Overall Record',
  'type': 'total',
  'displayValue': '7-3-2'}]

In [8]:
with open('../df_2023_LMX.csv', 'r', encoding='utf-8') as file:
    df_equipos = pd.read_csv(file)
    #df_equipos = df_equipos.iloc[:,[0]]
    for i, row in df_equipos.iterrows():
        print(row['ID'])
    


234
232
216
227
231
219
220
218
229
15720
233
20702
228
225
223
222
10125
17851


In [9]:
with open('../Historial_2024_LGM.csv', 'r', encoding='utf-8') as file:
    df_partidos = pd.read_csv(file)

In [10]:
# Restaurar las opciones a su valor por defecto si lo necesitas
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', 110)
df_partidos = df_partidos.drop_duplicates()
df_partidos = df_partidos.sort_values(by=['Home', 'Fecha'], ascending=[True, False], ignore_index=True)
df_partidos['Gana Local'] = None
df_partidos

,Fecha,H-Record,Home,H-Score,A-Score,Away,A-Record,Gana Local
0,2024-10-20,5-2-5,América,3.0,0.0,Santos,2-2-8,None
1,2024-09-30,4-1-5,América,0.0,1.0,Pumas UNAM,6-1-3,None
2,2024-09-18,4-0-4,América,3.0,0.0,Atlas,4-2-2,None
3,2024-09-15,3-0-4,América,1.0,0.0,Guadalajara,3-2-2,None
4,2024-08-25,2-0-3,América,0.0,1.0,Puebla,2-1-3,None
...,...,...,...,...,...,...,...,...
105,2024-09-28,6-3-1,Toluca,4.0,1.0,Atlas,4-2-4,None
106,2024-09-14,5-2-0,Toluca,4.0,0.0,Tijuana,4-2-2,None
107,2024-08-25,3-2-0,Toluca,2.0,1.0,Atlético de San Luis,2-2-2,None
108,2024-07-18,2-1-0,Toluca,3.0,0.0,Mazatlán FC,0-1-2,None


In [11]:
puntos_h, puntos_a = [], []
for i, equipo in df_partidos.iterrows():
    split_h = (equipo['H-Record']).split('-')
    split_a = (equipo['A-Record']).split('-')
    puntos_h.append(int(split_h[0]) + int(split_h[1]))
    puntos_a.append(int(split_a[0]) + int(split_a[1]))

df_partidos['Pts-H'] = puntos_h
df_partidos['Pts-A'] = puntos_a

df_partidos



,Fecha,H-Record,Home,H-Score,A-Score,Away,A-Record,Gana Local,Pts-H,Pts-A
0,2024-10-20,5-2-5,América,3.0,0.0,Santos,2-2-8,None,7,4
1,2024-09-30,4-1-5,América,0.0,1.0,Pumas UNAM,6-1-3,None,5,7
2,2024-09-18,4-0-4,América,3.0,0.0,Atlas,4-2-2,None,4,6
3,2024-09-15,3-0-4,América,1.0,0.0,Guadalajara,3-2-2,None,3,5
4,2024-08-25,2-0-3,América,0.0,1.0,Puebla,2-1-3,None,2,3
...,...,...,...,...,...,...,...,...,...,...
105,2024-09-28,6-3-1,Toluca,4.0,1.0,Atlas,4-2-4,None,9,6
106,2024-09-14,5-2-0,Toluca,4.0,0.0,Tijuana,4-2-2,None,7,6
107,2024-08-25,3-2-0,Toluca,2.0,1.0,Atlético de San Luis,2-2-2,None,5,4
108,2024-07-18,2-1-0,Toluca,3.0,0.0,Mazatlán FC,0-1-2,None,3,1


In [45]:
# Definimos las variables predictorias
X = df_partidos[['H-Score', 'A-Score', 'Pts-H', 'Pts-A']]

# Definir la variable objetivo
y = df_partidos['Gana Local']



In [42]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [46]:
# Dividir los datos en 70% para entrenamiento y 30% para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [49]:
# Crear el modelo de regresión logística
model = LogisticRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

ValueError: Unknown label type: unknown. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.